Sources:
https://platform.openai.com/docs/guides/fine-tuning

In [2]:
import yaml
import random

In [4]:
with open('Cat_0_Data/data/Cat_0_commonexample.yaml') as f:
    data = yaml.safe_load(f)

In [3]:
def parse_examples(examples: str) -> list[str]:
    """Converts examples from YAML file to list of utterance texts

    Args:
        examples (string): Examples from YAML data file.
                           It is a single string containing all utterances
                           separated by '\n- '

    Returns:
        List[str]: list of utterance texts
    """

    texts = [text.strip() for text in examples.lstrip("- ").split("\n- ")]
    # splitting by '\n -' causes empty string to be present in the list,
    # in case the value of examples is ''.
    if "" in texts:
        texts.remove("")
    return texts

A prompt separator string needs to be added at the end of the prompt both while fine tuning and sending requests to the model. Else, the outputted completions would most likely be random, instead of our desired output intents. This separator should NOT be present in the utterance at all.

In [4]:
prompt_separator = "\n\nIntent:\n\n"

Completions should start with a whitespace since OpenAI's tokenization tokenizes most words with a preceding whitespace.

For classification tasks, OpenAI recommends us to choose classes which map to a single token and set max_tokens=1. This is also a must when we want the model to output classification metrics. But some classes like "Enquiry" or "Incident" map to multiple tokens. <br>
Usually, we set a stop sequence at the end of the completion instead. But this won't work if we want to calculate classification metrics. So we convert the intent to strings of one token.

In [ ]:
intent_completion_mapping = {
    'Incident': ' Incident',
    'Request': ' Request',
    'Enquiry': ' En'
}

In [9]:
train_list = []
val_list = []

for intent_dict in data['nlu']:
    intent = intent_dict['intent']
    examples = parse_examples(intent_dict['examples'])
    random.shuffle(examples)

    train_set_size = int(0.8 * len(examples))
    
    for example in examples[:train_set_size]:
        train_list.append({'prompt': f'{example}{prompt_separator}', 
                          'completion': intent_completion_mapping[intent]})
    for example in examples[train_set_size:]:
        val_list.append({'prompt': f'{example}{prompt_separator}', 
                          'completion': intent_completion_mapping[intent]})

In [11]:
len(train_list), len(val_list)

(4349, 1089)

In [12]:
import jsonlines

In [13]:
with jsonlines.open('data_train.jsonl', mode='w') as writer:
    # Write each data item as a separate line
    for item in train_list:
        writer.write(item)

In [14]:
with jsonlines.open('data_val.jsonl', mode='w') as writer:
    # Write each data item as a separate line
    for item in val_list:
        writer.write(item)

In [5]:
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())

True

In [16]:
!openai api fine_tunes.create \
  -t  data_train.jsonl\
  -v data_val.jsonl \
  -m ada \
  --compute_classification_metrics \
  --classification_n_classes 3


Upload progress: 100%|███████████████████████| 560k/560k [00:00<00:00, 541Mit/s]
Uploaded file from data_train.jsonl: file-Rv7Dzp9MFx7xUlKZ0UJKKuy6
Upload progress: 100%|███████████████████████| 140k/140k [00:00<00:00, 115Mit/s]
Uploaded file from data_val.jsonl: file-LAL3AwKQ6mFR8Ms1RMMbNfrH
Created fine-tune: ft-nzoSpioROVwjAgEktgSHotLD
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-07-03 21:55:26] Created fine-tune: ft-nzoSpioROVwjAgEktgSHotLD

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-nzoSpioROVwjAgEktgSHotLD



In [ ]:
# !openai api fine_tunes.follow -i ft-9qk5AZqhQ2MJ78IZDMg7MDx0
# !openai api fine_tunes.follow -i ft-JLUz1rNg4bKu9Vr18DdeXidx
!openai api fine_tunes.follow -i ft-nzoSpioROVwjAgEktgSHotLD

[2023-07-03 21:55:26] Created fine-tune: ft-nzoSpioROVwjAgEktgSHotLD


In [82]:
# !openai api fine_tunes.results -i ft-9qk5AZqhQ2MJ78IZDMg7MDx0
!openai api fine_tunes.results -i ft-JLUz1rNg4bKu9Vr18DdeXidx

step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy,validation_loss,validation_sequence_accuracy,validation_token_accuracy,classification/accuracy,classification/weighted_f1_score
1,520,8,0.3662363214270124,0.0,0.0,0.6415449769689349,0.0,0.0,,
2,1168,16,0.2698770616489669,0.0,0.0,,,,,
3,1496,24,0.4214884167602067,0.0,0.0,,,,,
4,1952,32,0.21759486844362594,0.0,0.0,,,,,
5,2344,40,0.13543927974877382,0.125,0.125,,,,,
6,2800,48,0.06393603897911816,0.625,0.625,,,,,
7,3064,56,0.07467400160054238,0.625,0.625,,,,,
8,3392,64,0.07126223466201918,0.5,0.5,,,,,
9,3912,72,0.040091332210267866,0.5,0.5,0.07438962268139221,0.25,0.25,,
10,4496,80,0.030143699689813525,0.5,0.5,,,,,
11,4824,88,0.06904414493069636,0.25,0.25,,,,,
12,5216,96,0.0721331408067878,0.125,0.125,,,,,
13,5480,104,0.047779257237489214,0.75,0.75,,,,,
14,5872,112,0.04085173117743882,0.75,0.75,,,,,
15,6392,120,0.030056229201836683,0.875,0.875,,,,,
16,7232,128,0.02556421588052147,0.375,0.3

In [84]:
import openai
response = openai.Completion.create(
    model="ada:ft-personal-2023-06-28-10-24-36",
    prompt=f"I got logged out of my account because of MFA{prompt_separator}",
    max_tokens=1
)
print(response['choices'][0]['text'].strip())

Incident
